# pyBOS demonstration notebook



## imports
This notebook is not intented to be a lecture about BOS but only a demonstration
of what can be done with the library pyBOS.

First we import packages that we'll use later.
`%matplotlib widget` is here for improving plots quality in this notebook

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path

%matplotlib widget

Then we import stuff fron the pyBOS library

In [ ]:
from pybos.extraction import idz2n
from pybos.laplacian import solve_BOS
from pybos.physical_laws import polynomial
from pybos.reader import DANTEC_DAT, reader
from pybos.vector_calculus import amplitude, divergence

## Reading the data

We use the `reader` function to read the data.
It must be a `.csv` type file (including dome of the tecplot's `.plt.` files)

If possible prefer using the calibration from Dantec/Davis and avoid importing data in pixel.

In [ ]:
im_datafile = Path("data/example_data_2d.dat")
cols = 2, 3, 6, 7  # pix for now
scale = 1 / (2560 * ((283 - 26) / 319) / 384)  # pix to mm

data = reader(im_datafile, DANTEC_DAT, usecols=cols, scale=scale, unit="mm", output_unit="mm")

reverse = True  # if reference image and final image are reversed (should not happen !)
if reverse:
    data["U"] *= -1
    data["V"] *= -1

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))
data.U.plot.pcolormesh(ax=ax1, cmap=plt.cm.jet)
data.V.plot.pcolormesh(ax=ax2, cmap=plt.cm.jet)
ax1.axis("equal")
ax2.axis("equal")
fig.tight_layout()
plt.show()

data

## Preparing the data

You may want to crop the data to remove reflection and other defects.
Here we are only using the `isel` method of `Xarray`'s `Dataset`.

In [ ]:
roi = {"x": slice(None, None), "y": slice(None, None)}

crop = True
if crop:
    pix1, piy1 = 244, 117
    pix2, piy2 = 2321, 2012
    Pix_size = 8

    pix1 = int((pix1 - Pix_size + 0.5) / Pix_size)
    pix2 = int((pix2 - Pix_size + 0.5) / Pix_size)
    piy1 = int((piy1 - Pix_size + 0.5) / Pix_size)
    piy2 = int((piy2 - Pix_size + 0.5) / Pix_size)

    piy1, piy2 = data.dims["y"] - piy2, data.dims["y"] - piy1

    roi = {"x": slice(pix1, pix2), "y": slice(piy1, piy2)}

cropped_data = data.isel(roi)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))
data.U.plot.pcolormesh(ax=ax1, cmap=plt.cm.jet)
(cropped_data.U * 0.0).plot.pcolormesh(ax=ax1, alpha=0.5, add_colorbar=False)
data.V.plot.pcolormesh(ax=ax2, cmap=plt.cm.jet)
(cropped_data.V * 0.0).plot.pcolormesh(ax=ax2, alpha=0.5, add_colorbar=False)
ax1.axis("equal")
ax2.axis("equal")
fig.tight_layout()
plt.show()

cropped_data

You may also want to filter the data. Here we are only using standard filter from `scipy`.

In order to keep the `Xarray`'s `Dataset` data format, we use the `apply_ufunc` wrapper.

In [ ]:
filt_method = "gaussian"
if filt_method == "gaussian":
    from scipy.ndimage import gaussian_filter

    sigma = 2, 2
    filter_ = lambda x: gaussian_filter(x, sigma=sigma, order=0, mode="nearest")

elif filt_method == "spline":
    from scipy.signal import spline_filter

    lmbda = 2
    filter_ = lambda x: spline_filter(x, lmbda=lmbda)

filtered_data = xr.apply_ufunc(filter_, cropped_data, keep_attrs=True)

filtered_data

There is a function for computing the displacement's amplitude and divergence.

In [ ]:
filtered_data["amplitude"] = amplitude(filtered_data)
filtered_data["divergence"] = divergence(filtered_data)

print(f"mean(amplitude)={float(filtered_data.amplitude.mean())}")
print(f"mean(divergence)={float(filtered_data.divergence.mean())}")

fig, axs = plt.subplots(2, 2, figsize=(9, 9))
filtered_data.U.plot.pcolormesh(ax=axs[0][0], cmap=plt.cm.jet)
filtered_data.V.plot.pcolormesh(ax=axs[0][1], cmap=plt.cm.jet)
filtered_data.amplitude.plot.pcolormesh(ax=axs[1][0], cmap=plt.cm.jet)
filtered_data.divergence.plot.pcolormesh(ax=axs[1][1], cmap=plt.cm.jet)

for axx in axs:
    for axy in axx:
        axy.axis("equal")
fig.tight_layout()
plt.show()

## Resolution

Now we define the properties of the experiment (size of the tank, boundary conditions)
and the physical law for converting refracting index to to temperature and conversely.

In [ ]:
d = 190  # for cube # 0.003 for aquarium # [mm] pattern-tank distance
w = 400  # [mm] tank width "h" in [Plakina et al 2012]
n0 = 1.00027  # [] refractive index of the air

n_T, T_n = polynomial()

T_left = 7.5
T_right = 39.5

idz_top = idz_bottom = None
idz_left = n_T(T_left) * w
idz_right = n_T(T_right) * w

And compute the refractive index

In [ ]:
idz = solve_BOS(filtered_data.divergence, w, d, n0, idz_top, idz_bottom, idz_left, idz_right)

fig, ax = plt.subplots()
idz.plot.pcolormesh(ax=ax, cmap=plt.cm.jet)
ax.axis("equal")
fig.tight_layout()
plt.show()

idz

$n$ can be extracted using a 2d hypothesis

In [ ]:
n = idz2n(idz, w, method="2d")

fig, ax = plt.subplots()
n.plot.pcolormesh(ax=ax, cmap=plt.cm.jet)
ax.axis("equal")
fig.tight_layout()
plt.show()

n

That we can convert to temperature using the aformentionned law

In [ ]:
T = T_n(n)

fig, ax = plt.subplots()
T.plot.pcolormesh(ax=ax, cmap=plt.cm.jet, vmin=0, vmax=50)
ax.axis("equal")
fig.tight_layout()
plt.savefig("Temperature_2d.png", dpi=200)

T

Now it is your turn to use the library ;) !